# 🚀 How do I send a API requests?

1. How can I reverse engineer 'the correct' API request when there is no documentation?

   - Monitor network traffic in the browser
   - use the `log-api` command in [Domo Java CLI](https://domo-support.domo.com/s/article/360043437733?language=en_US)
   - see [✨ Jae Wilsons TreasureTrove](./README.md#helpful-links)
<br>

2. How does the internet handle authentication?
   - pass authentication in cookies
   - pass authentication via request headers
       - different APIs support different authentication schemes

## ▶️ How to Monitor Network Traffic

1. Go to [Domo > Data > Accounts](https://domo-community.domo.com/datacenter/accounts) to view a list of all accounts you have access to.

2. To Monitor network traffic > Inspect > Network.  

3. Find the API request.
   - Filter for "providers".
   - Examine the URL, headers, request Method (GET, PUT, POST, DELETE) and body<br>
   - Copy the query into a text editor.


### 3 ways to Authenticate the same API request

### ▶️🍪 convert an API request into a function

> Functions improve code legibility, recycle-ability, and maintainability by being *callable* and *parameterized*

1. modify the starter function below receive `headers: dict` and modify the request.
```
headers = {'cookie': <your_header>}
```
2. Use the `requests` library to send an API request to the providers endpoint.
3. test by scraping your browser cookie and passing it into `headers`

4. copy your finished function into a new file `./functions/accounts.py`.

5. modify your test to import your function from the module

```
from functions.accounts import get_accounts()
```

In [2]:
# uncomment to install the requests library
# %pip install requests

In [7]:
import requests

# FIX ME -- then move me into ./functions/accounts.py
def get_accounts(domo_instance, debug_api: bool = False):
    url = f"https://{domo_instance}.domo.com/api/data/v2/datasources/providers"
    
    method = 'get'
    
    if debug_api:
        print({"url": url,
               "method" : method,
               "headers" : headers})
        
    return requests.request(method=method, url=url)

#### Test Implementation of get_accounts

In [10]:
# from solutions.get_accounts_v1 import get_accounts

DOMO_INSTANCE = (
    "domo-community"  ## it's common practice to declare constants using all caps.
)
COOKIE = "cookie_goes_here"

# fix me
headers = {}

res = get_accounts(domo_instance=DOMO_INSTANCE,
                   # headers = headers,
                   debug_api = False)
res

<Response [401]>

### 🎓 What is <Response [401]>

1. requests.request [returns an instance of the Response class](https://www.geeksforgeeks.org/response-request-python-requests/) || `requests.models.Response`

2. classes are `dict` that have attributes (fields) and methods (functions)
   - ex. the `Response` class has attributes like `status_code` and method `.json()`

In [16]:
print(type(res))
# print(vars(res)) # prints attributes of res 


# from the error message it's clear we need to pass authentication with our API request
print({"status": res.status_code, "is_success": res.ok,})

print("\n")
type(res.json()) # prints the type of the result of res.json() 
# print("\n")
res.json()


<class 'requests.models.Response'>
{'status': 401, 'is_success': False}




{'status': 401,
 'statusReason': 'Unauthorized',
 'path': '/api/data/v2/datasources/providers',
 'message': 'Full authentication is required to access this resource',
 'toe': 'BRLYX19LG7-3RNAP-UI99Z'}

### ▶️ modify `get_accounts` to return a list of account_objects

The `requests.models.Response` class has a method (function), `.json()` that will convert the response into a dictionary.


### 🧪 use tests to communicate assumptions 
- How might we test for API errors?
- How might we test for 'logic errors' ex. No accounts returned?

## 🚀 Use a username and password authentication flow to handle API Authentication

## 🎓 "full authentication" is not the same as client_id and secret authentication

Client_ID and Secret (AKA developer_token authentication) is for public APIs  as documented under <https://developer.domo.com>.  

For more information [Get Outta the UI and into APIs - Domo IDEA Exchange 2022](https://www.youtube.com/watch?v=hRwrZABP8RE).

### ▶️🤐 implement a function, `get_session_token` to handle username and password authentication
- function should receive: `domo_instance: str, domo_username: str, domo_password : str, and return_raw: bool = False`

- `get_session_token` should parse the response and return just the `sessionToken`

- store the final function in `./functions/auth.py`

🧪 how might we implement tests to ensure expected / assumed behavior
- can you test for 400 - 500 errors? (res.ok)
- what if you get a 200 response but sent an invalid password? 

💡 to support debugging, add a parameter `return_raw` before any tests
```
if return_raw: return res
```
- this helps debugging when there's a gap between expected vs actual API behavior



In [18]:
# move into ./functions/auth.py
def get_session_token(domo_instance: str, 
                      # add parameters here
                      return_raw: bool = False) -> str:

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    # fix me
    body = {
        "method": "password",
        "emailAddress": "domo_username",
        "password": "domo_password",
    }

    res = requests.request(method="POST", url=url, json=body, verify=False)

    if return_raw:
        return res
    
    # logic tests go here

    return res

In [7]:
# from solutions.get_session_token import get_session_token

# these are 'real creds'
get_session_token(
    domo_instance = 'domo-community',
    domo_username = 'dp24@test.com',
    domo_password = 'thisisinsecure',
)

get_session_token() got an unexpected keyword argument 'domo_username'


### ▶️ modify get_accounts() to optionally receive session_token.

1. modify get_accounts to receive `headers: dict = None, session_token : str = None`

2. combine headers and session_token and pass to request

```
# handle if users don't pass headers
headers = headers or {} 

 # use the .update() method to update a dictionary
headers.update({"session_token": session_token})
```

3. retrieve `session_token` from `get_full_auth()` then pass it to your modified `get_accounts`

[solution](./solutions/get_accounts_v2.py)

### ▶️ Put it all together!

In [ ]:
# copy your implementation of get_accounts from abvoe
# modify, and test using sample below
# update your source code in ./functions.accounts.py

In [19]:
session_token = get_session_token(
    domo_instance = 'domo-community',
    # domo_username = 'dp24@test.com',
    # domo_password = 'thisisinsecure',
)


get_accounts(domo_instance=DOMO_INSTANCE,
            #  session_token=session_token
             )

/home/domo/.conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domo-community.domo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [401]>

## 🧪 Extra Credit - uses layers and classes to introduce consistency to your code

Let's envision our codebase as having three layers. 

| Layer / Class     | Returns | Calls | Logic
| -------- | ------- | ------- |------- |
|api_function |ResponseClass| calls API directly | Just calls APIs and tests for Errors
|DomoClass| DomoClass| calls API_Functions | no implementation specific logic 
|implementation_function|Log | calls class_functions or implementation_function | one time use

### separating API calls from implementation protects us from API changes / versioning
We separate API functions from Class functions so when Domo changes the API (from V1 to V2) we just have to implement a new API function, and call a different function in the class_function.  


Implementing class functions is outside the scope of this course, but this design pattern was used in [DomoLibrary](https://jaewilson07.github.io/domolibrary/)<br>
[Route Functions for Accounts](https://github.com/jaewilson07/domolibrary/blob/main/domolibrary/routes/account.py)<br>
[Class Implementation of Accounts](https://github.com/jaewilson07/domolibrary/blob/main/domolibrary/classes/DomoAccount.py)

<br>


- create a file `./functions/client.py` 
- create a `@dataclass ResponseClass` with attributes `status: int, is_success: bool , response: Union[dict, str]` 

- add classmethod that converts a `requests.models.Response` into an instance of `ResponseClass`

- modify `get_accounts` to return `ResponseClass`
 
This ensures that each `api_function` always returns the same format response (which simplifies testing downstream) even if you switch do a different library (like HTTPX).

Note: To keep with the `api_Function` design pattern, we could convert `get_session_token()` to return a `ResponseClass` where the response has been modified to return just the session_token

[Solution](./solutions/client.py)

In [21]:
# from solutions.client import ResponseClass

from dataclasses import dataclass

# finish me and move to ./functions/client.py
@dataclass
class ResponseClass:
    status: int
    # parameters go here.

    @classmethod
    def from_request_response(cls, res: requests.models.Response):
        return cls(
            status = res.status_code
            #  more parameters go here
            )
        



In [22]:
ResponseClass.from_request_response(res)

ResponseClass(status=401)

## 🚀 Solution

In [25]:
# from solutions.accounts import get_accounts
# from solutions.auth import get_session_token


session_token = get_session_token(
    domo_instance = 'domo-community',
    # domo_username = 'dp24@test.com',
    # domo_password = 'thisisinsecure',
)

get_accounts(domo_instance=DOMO_INSTANCE,
                # session_token=session_token,
                debug_api= False
                )


/home/domo/.conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domo-community.domo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [401]>